# Titanic Survival Prediction - Example Notebook

This notebook demonstrates the complete ML workflow for the mlsys project:
1. Load and explore data
2. Perform feature engineering
3. Train and evaluate a classification model
4. Upload trained model to GCS

**Model**: Titanic survival prediction (binary classification)

**Data Source**: sklearn's OpenML Titanic dataset (for demonstration purposes)

**Model Version**: v1

## Setup

Import required libraries and configure environment.

In [ ]:
# Standard library
import json
import warnings
from datetime import UTC, datetime

# Third-party libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_openml
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    roc_auc_score,
    roc_curve,
)
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler

# mlsys utilities
from mlsys.gcs import gcs_put
from mlsys.settings import GCS_BUCKET_MODELS_DEV
from mlsys.vis import setup_plot_style

# Configure
warnings.filterwarnings("ignore")
setup_plot_style()
pd.set_option("display.max_columns", None)

print("Setup complete!")

## 1. Load Data

For this example, we'll use sklearn's Titanic dataset. In production, you would use `bq_get()` to fetch data from BigQuery:

```python
from mlsys.bq import bq_get

df = bq_get("""
    SELECT * 
    FROM `my-project.my_dataset.titanic_training`
    WHERE date >= '2024-01-01'
""")
```

In [ ]:
# Load Titanic dataset from sklearn
titanic = fetch_openml("titanic", version=1, as_frame=True, parser="auto")
df = titanic.frame

print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")
df.head()

## 2. Exploratory Data Analysis (EDA)

### 2.1 Data Overview

In [ ]:
# Basic info
print("Dataset Info:")
print(df.info())
print("\n" + "=" * 50 + "\n")

# Summary statistics
print("Summary Statistics:")
df.describe()

In [ ]:
# Missing values
print("Missing Values:")
missing = df.isnull().sum()
missing_pct = (missing / len(df)) * 100
missing_df = pd.DataFrame({"Count": missing, "Percentage": missing_pct})
missing_df[missing_df["Count"] > 0].sort_values("Count", ascending=False)

### 2.2 Target Variable Distribution

In [ ]:
# Survival distribution
survival_counts = df["survived"].value_counts()
print("Survival Counts:")
print(survival_counts)
print(f"\nSurvival Rate: {(survival_counts[1] / len(df)) * 100:.2f}%")

# Visualize
fig, ax = plt.subplots(1, 2, figsize=(12, 4))

survival_counts.plot(kind="bar", ax=ax[0])
ax[0].set_title("Survival Counts")
ax[0].set_xlabel("Survived")
ax[0].set_ylabel("Count")
ax[0].set_xticklabels(["No", "Yes"], rotation=0)

survival_counts.plot(kind="pie", autopct="%1.1f%%", ax=ax[1], labels=["Died", "Survived"])
ax[1].set_title("Survival Rate")
ax[1].set_ylabel("")

plt.tight_layout()
plt.show()

### 2.3 Feature Analysis

In [ ]:
# Survival by passenger class
survival_by_class = df.groupby("pclass")["survived"].apply(lambda x: (x == "1").mean())

fig, ax = plt.subplots(1, 3, figsize=(15, 4))

# By class
survival_by_class.plot(kind="bar", ax=ax[0])
ax[0].set_title("Survival Rate by Passenger Class")
ax[0].set_xlabel("Class")
ax[0].set_ylabel("Survival Rate")
ax[0].set_xticklabels(["1st", "2nd", "3rd"], rotation=0)

# By sex
survival_by_sex = df.groupby("sex")["survived"].apply(lambda x: (x == "1").mean())
survival_by_sex.plot(kind="bar", ax=ax[1])
ax[1].set_title("Survival Rate by Sex")
ax[1].set_xlabel("Sex")
ax[1].set_ylabel("Survival Rate")
ax[1].set_xticklabels(["Female", "Male"], rotation=0)

# Age distribution by survival
df[df["survived"] == "1"]["age"].hist(bins=30, alpha=0.5, label="Survived", ax=ax[2])
df[df["survived"] == "0"]["age"].hist(bins=30, alpha=0.5, label="Died", ax=ax[2])
ax[2].set_title("Age Distribution by Survival")
ax[2].set_xlabel("Age")
ax[2].set_ylabel("Count")
ax[2].legend()

plt.tight_layout()
plt.show()

## 3. Feature Engineering

Prepare features for modeling.

In [ ]:
# Create a copy for feature engineering
df_model = df.copy()

# Convert target to numeric
df_model["survived"] = df_model["survived"].astype(int)

# Select features for modeling
# Using features that are most predictive and have reasonable completeness
feature_cols = ["pclass", "sex", "age", "sibsp", "parch", "fare"]

# Create feature matrix and target
X = df_model[feature_cols].copy()
y = df_model["survived"]

print(f"Feature shape: {X.shape}")
print(f"Target shape: {y.shape}")
print(f"\nFeatures: {list(X.columns)}")

In [ ]:
# Handle missing values
print("Missing values before imputation:")
print(X.isnull().sum())

# Impute age with median
X["age"] = X["age"].fillna(X["age"].median())

# Impute fare with median
X["fare"] = X["fare"].fillna(X["fare"].median())

print("\nMissing values after imputation:")
print(X.isnull().sum())

In [ ]:
# Encode categorical variables
X["sex"] = X["sex"].map({"male": 0, "female": 1})

print("Feature matrix after encoding:")
X.head()

In [ ]:
# Remove any remaining rows with missing values
mask = X.isnull().any(axis=1) | y.isnull()
X = X[~mask]
y = y[~mask]

print(f"Final dataset shape: {X.shape}")
print(f"Removed {mask.sum()} rows with missing values")

## 4. Train/Test Split

In [ ]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set size: {X_train.shape}")
print(f"Test set size: {X_test.shape}")
print("\nTarget distribution in training set:")
print(y_train.value_counts(normalize=True))

## 5. Feature Scaling

Scale numerical features to improve model performance.

In [ ]:
# Scale features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Convert back to DataFrames for convenience
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns, index=X_train.index)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns, index=X_test.index)

print("Features scaled successfully!")
print("\nScaled training data (first 5 rows):")
X_train_scaled.head()

## 6. Model Training

Train a Random Forest classifier.

In [ ]:
# Train model
model = RandomForestClassifier(
    n_estimators=100, max_depth=10, min_samples_split=5, random_state=42, n_jobs=-1
)

print("Training model...")
model.fit(X_train_scaled, y_train)
print("Model trained successfully!")

# Feature importance
feature_importance = pd.DataFrame(
    {"feature": X_train.columns, "importance": model.feature_importances_}
).sort_values("importance", ascending=False)

print("\nFeature Importance:")
print(feature_importance)

In [ ]:
# Visualize feature importance
plt.figure(figsize=(10, 6))
plt.barh(feature_importance["feature"], feature_importance["importance"])
plt.xlabel("Importance")
plt.ylabel("Feature")
plt.title("Feature Importance")
plt.tight_layout()
plt.show()

## 7. Model Evaluation

Evaluate model performance on the test set.

In [ ]:
# Predictions
y_pred = model.predict(X_test_scaled)
y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]

# Metrics
accuracy = accuracy_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"Test Accuracy: {accuracy:.4f}")
print(f"Test ROC AUC: {roc_auc:.4f}")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=["Died", "Survived"]))

In [ ]:
# Confusion matrix
cm = confusion_matrix(y_test, y_pred)

fig, ax = plt.subplots(1, 2, figsize=(12, 4))

# Confusion matrix
im = ax[0].imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
ax[0].figure.colorbar(im, ax=ax[0])
ax[0].set(
    xticks=np.arange(cm.shape[1]),
    yticks=np.arange(cm.shape[0]),
    xticklabels=["Died", "Survived"],
    yticklabels=["Died", "Survived"],
    title="Confusion Matrix",
    ylabel="True label",
    xlabel="Predicted label",
)

# Add text annotations
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        ax[0].text(j, i, format(cm[i, j], "d"), ha="center", va="center", color="white")

# ROC curve
fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
ax[1].plot(fpr, tpr, label=f"ROC curve (AUC = {roc_auc:.4f})")
ax[1].plot([0, 1], [0, 1], "k--", label="Random classifier")
ax[1].set_xlabel("False Positive Rate")
ax[1].set_ylabel("True Positive Rate")
ax[1].set_title("ROC Curve")
ax[1].legend()
ax[1].grid(True)

plt.tight_layout()
plt.show()

## 8. Cross-Validation

Verify model performance with cross-validation.

In [ ]:
# Cross-validation on full dataset
X_scaled = scaler.fit_transform(X)
cv_scores = cross_val_score(model, X_scaled, y, cv=5, scoring="roc_auc")

print(f"Cross-validation ROC AUC scores: {cv_scores}")
print(f"Mean ROC AUC: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")

## 9. Upload Model to GCS

Save the trained model and scaler to Google Cloud Storage for deployment.

**Model artifacts**:
- `model.pkl`: Trained RandomForest classifier
- `scaler.pkl`: StandardScaler fitted on training data

**GCS path**: `gs://ml-models-dev/titanic-survival/v1/`

In [ ]:
# Define model metadata
MODEL_NAME = "titanic-survival"
MODEL_VERSION = "v1"
BUCKET_NAME = GCS_BUCKET_MODELS_DEV

print(f"Model: {MODEL_NAME}")
print(f"Version: {MODEL_VERSION}")
print(f"Bucket: {BUCKET_NAME}")

In [ ]:
# Upload model artifact
model_path = f"{MODEL_NAME}/{MODEL_VERSION}/model.pkl"
print(f"Uploading model to gs://{BUCKET_NAME}/{model_path}")

gcs_put(obj=model, bucket_name=BUCKET_NAME, blob_path=model_path)
print("Model uploaded successfully!")

In [ ]:
# Upload scaler
scaler_path = f"{MODEL_NAME}/{MODEL_VERSION}/scaler.pkl"
print(f"Uploading scaler to gs://{BUCKET_NAME}/{scaler_path}")

gcs_put(obj=scaler, bucket_name=BUCKET_NAME, blob_path=scaler_path)
print("Scaler uploaded successfully!")

## 10. Model Metadata

Document model metadata for tracking and reproducibility.

In [ ]:
# Model metadata
metadata = {
    "model_name": MODEL_NAME,
    "model_version": MODEL_VERSION,
    "model_type": "RandomForestClassifier",
    "training_date": datetime.now(UTC).isoformat(),
    "features": list(X.columns),
    "target": "survived",
    "training_samples": len(X_train),
    "test_samples": len(X_test),
    "test_accuracy": float(accuracy),
    "test_roc_auc": float(roc_auc),
    "cv_roc_auc_mean": float(cv_scores.mean()),
    "cv_roc_auc_std": float(cv_scores.std()),
    "hyperparameters": {
        "n_estimators": 100,
        "max_depth": 10,
        "min_samples_split": 5,
        "random_state": 42,
    },
    "gcs_path": f"gs://{BUCKET_NAME}/{MODEL_NAME}/{MODEL_VERSION}/",
}

print("Model Metadata:")
for key, value in metadata.items():
    print(f"  {key}: {value}")

In [ ]:
# Save metadata to GCS
metadata_path = f"{MODEL_NAME}/{MODEL_VERSION}/metadata.json"
print(f"Uploading metadata to gs://{BUCKET_NAME}/{metadata_path}")

gcs_put(obj=json.dumps(metadata, indent=2), bucket_name=BUCKET_NAME, blob_path=metadata_path)
print("Metadata uploaded successfully!")

## Summary

### Model Performance
- **Test Accuracy**: {accuracy:.4f}
- **Test ROC AUC**: {roc_auc:.4f}
- **CV ROC AUC**: {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})

### Artifacts Uploaded to GCS
1. **Model**: `gs://ml-models-dev/titanic-survival/v1/model.pkl`
2. **Scaler**: `gs://ml-models-dev/titanic-survival/v1/scaler.pkl`
3. **Metadata**: `gs://ml-models-dev/titanic-survival/v1/metadata.json`

### Next Steps
1. Create an Airflow DAG to schedule predictions
2. Deploy to Cloud Run for production
3. Monitor model performance over time
4. Iterate on features and hyperparameters for improved performance